* Toma el dataset original (raw) y usando PySpark realiza los primeros filtros porque contiene 330.574 filas y 369 columnas
* Guarda un df pandas con los años 2016, 2017 y 2018

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#verificar versión de spark en : https://spark.apache.org/downloads.html

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
findspark.init()

In [4]:
#este código sirve para forzar la instalación de una versión espécífica de Java
import os #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" #set environment variable
  !java -version #check java version

install_java()

openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [5]:
#Para verificar versión de Java
!java -version
!sudo update-alternatives --config java
!java -version

openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 
openjdk version "11.0.7" 2020-04-14
OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04)
OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)


In [6]:
#sesion de spark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()

In [7]:
import numpy as np
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import os


In [8]:
#leer archivo
sdf0 = spark.read.csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Raw/encuesta_calidad_vida.csv', inferSchema = True, header = True, sep= ';' )

In [9]:
sdf0.count(), len(sdf0.columns)

(330574, 369)

In [10]:
#esquema
sdf0.printSchema()

root
 |-- encuesta_calidad.encuesta: double (nullable = true)
 |-- encuesta_calidad.barrio: string (nullable = true)
 |-- encuesta_calidad.comuna: string (nullable = true)
 |-- encuesta_calidad.estrato: integer (nullable = true)
 |-- encuesta_calidad.hogar: integer (nullable = true)
 |-- encuesta_calidad.persona: integer (nullable = true)
 |-- encuesta_calidad.factor vivienda: double (nullable = true)
 |-- encuesta_calidad.factor persona: double (nullable = true)
 |-- encuesta_calidad.año: integer (nullable = true)
 |-- encuesta_calidad.p_146: integer (nullable = true)
 |-- encuesta_calidad.p_147: integer (nullable = true)
 |-- encuesta_calidad.p_148: integer (nullable = true)
 |-- encuesta_calidad.p_149: integer (nullable = true)
 |-- encuesta_calidad.p_150: integer (nullable = true)
 |-- encuesta_calidad.p_151: integer (nullable = true)
 |-- encuesta_calidad.p_152: integer (nullable = true)
 |-- encuesta_calidad.p_153: integer (nullable = true)
 |-- encuesta_calidad.p_154: integer (n

In [11]:
#renombrar columnas
new_column_name_list= list(map(lambda x: x.replace("encuesta_calidad.", ""), sdf0.columns))
sdf0 = sdf0.toDF(*new_column_name_list)

In [12]:
#seleccionar algunas columnas
sdf0.select("encuesta","barrio","comuna","estrato") \
.show(15, truncate=False) #the truncate=False parameter that adjusts the size of columns to prevent values from being cut off.

+--------+------------------------+----------------+-------+
|encuesta|barrio                  |comuna          |estrato|
+--------+------------------------+----------------+-------+
|12012.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12012.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12012.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12016.0 |LAURELES                |LAURELES-ESTADIO|5      |
|12016.0 |LAURELES                |LAURELES-ESTADIO|5      |
|12016.0 |LAURELES                |LAURELES-ESTADIO|5      |
|12016.0 |LAURELES                |LAURELES-ESTADIO|5      |
|12016.0 |LAURELES                |LAURELES-ESTADIO|5      |
|12016.0 |LAURELES                |LAURELES-ESTADIO|5      |
|12017.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12017.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12017.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12017.0 |SANTO DOMINGO SABIO Nº 1|POPULAR         |2      |
|12018.0 |CAMPO VALDES N

In [13]:
# Agrupar y contar
sdf0.groupBy("año") \
.count() \
.orderBy("count", ascending=False) \
.show()


+----+-----+
| año|count|
+----+-----+
|2011|49339|
|2012|48453|
|2013|47901|
|2014|47341|
|2015|44484|
|2016|31876|
|2018|30934|
|2017|30246|
+----+-----+



In [14]:
#conservar años >= 2016
sdf1 = sdf0.filter(sdf0.año >= 2016)
sdf1.groupBy('año').count().show()

+----+-----+
| año|count|
+----+-----+
|2018|30934|
|2016|31876|
|2017|30246|
+----+-----+



In [15]:
#guardar archivo con cambios en las columnas y tres últimos años

In [16]:
sdf1.repartition(1).write.mode("overwrite").option("header", "true").csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Processed/T0_ECV.csv')

In [17]:
#leer archivo con pandas

In [19]:
df1 = pd.read_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Processed/T0_ECV.csv/part-00000-194248b9-f223-4211-84de-ea42f40d233a-c000.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (224,250) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
df1.shape

(93056, 369)

In [21]:
#guardarlo con pandas

In [23]:
df1.to_csv('/content/gdrive/My Drive/calidad_de_vida_medellin/github/Data/Processed/T0_ECV_161718.csv', sep=';')